# La pipeline : du Dataloader a l'entree du reseau

In [1]:
import numpy as np
import skimage
import torch
from PIL import Image
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection

ModuleNotFoundError: No module named 'torch'

## Data Loader
Le `Data Loader` est une classe de pytorch qui permet de charger les images du `Dataset` en retournant a chaque step un batch de (images, boxes, labels, difficulties).

Le `Dataset` lit les fichiers `.json`.

Ici, on se place dans le cas d'un batch d'une seule image (on n'a pas besoin du label ni de la difficulty).

In [2]:
# Epoch p, step n : donnees retournees par le dataloader dans train.py
img_path = '../img/sitting_salon.png'

image = Image.open(img_path, mode='r')
bbox = torch.Tensor([[16, 13, 34, 46], [3, 16, 15, 37]])
image.size, bbox

NameError: name 'Image' is not defined

# Preprocessing
Le preprocessing a lieu lorsque le `Dataloader` appel un batch d'image en utilisant la fonction `__getitem__()` du `Dataset`. 

**Chaque image est passée a travers la fonction `thermal_image_preprocessing()` dans `utils.py`**.

### Standardization
**On soustrait la moyenne et on divise par l'ecart type**. La moyenne et l'ecart type sont calcules sur tout le training set automatiquement grace a la fonction `dataset_mean_std()` de la classe Dataset.

Pour cet exemple, on utilise la moyenne et l'ecart type de notre unique image.

In [34]:
# PIL image to numpy array
image = np.array(image)
image = np.expand_dims(image, axis=-1)

mean, std = image.mean(), image.std()
image.shape, mean, std, image.min(), image.max()

((60, 80, 1), 29294.899166666666, 158.94683282774847, 29049, 30253)

In [53]:
image = (image - mean) / std
image = image.astype('float32')
image.mean(), image.std(), image.min(), image.max()

(2.5431316e-08, 1.0, -1.547053, 6.027807)

### Augmentation
L'image et la bbox associee sont passees dans la fonction `data_augmentation()` dans `utils.py`.

J'utilise la librairie [imgaug](https://imgaug.readthedocs.io/en/latest/).

In [54]:
# Creation de l'objet BoundingBoxesOnImage qui constitue la liste des bounding boxes de l'image
list_box = []

for box in bbox.tolist():
    list_box.append(BoundingBox(*box))
    
bbs = BoundingBoxesOnImage(list_box, shape=image.shape)
bbs

BoundingBoxesOnImage([BoundingBox(x1=16.0000, y1=13.0000, x2=34.0000, y2=46.0000, label=None), BoundingBox(x1=3.0000, y1=16.0000, x2=15.0000, y2=37.0000, label=None)], shape=(60, 80))

In [38]:
# On definie la liste des transformations a faire
augmenters = [iaa.SomeOf((1, 3),
                 [iaa.Crop(percent=(0.1, 0.2)),
                  iaa.OneOf([iaa.Dropout(p=(0.01, 0.2)),
                             iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05))]),
                  iaa.Fliplr(1.0)],random_order=True)]

In [55]:
def transformation(image, bbs):
    # On fixe une limite de 50 essais pour faire la transformation
    #(si une boxe se retrouve en dehors de l'image par exemple)
    for i in range(50):
            seq = iaa.Sometimes(0.8, augmenters)
            image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

            wrong_boxes = 0
            for bb in bbs_aug.bounding_boxes:
                bb_cliped_area = bb.clip_out_of_image(image_aug).area
                bb_area = bb.area
                # Si on crop plus de 20% d'une bbox lors du clip_out_of_image(), on recommence.
                if bb_cliped_area / bb_area < 0.8:
                    wrong_boxes += 1
            # Si toutes les bbox de l'images sont a l'interieur de l'image, on garde cet essai
            if wrong_boxes == 0:
                bbs_aug = bbs_aug.clip_out_of_image()
                break
    return image_aug, bbs_aug

In [56]:
# Simulation de 10 steps : a chacune d'elles, l'image est transformee aleatoirement (ou pas). 
for i in range(10):
    image_aug, bbs_aug = transformation(image, bbs)
    
image_aug.shape, bbs_aug

#TODO : visualisation

((60, 80),
 BoundingBoxesOnImage([BoundingBox(x1=16.0000, y1=13.0000, x2=34.0000, y2=46.0000, label=None), BoundingBox(x1=3.0000, y1=16.0000, x2=15.0000, y2=37.0000, label=None)], shape=(60, 80)))

In [57]:
# Pour recuperer la/les bounding box(es) transforme(es) sour la forme d'un tensor:
boxes = []
for nb in bbs_aug.bounding_boxes:
            boxes.append([nb.x1, nb.y1, nb.x2, nb.y2])
bbox = torch.FloatTensor(boxes)
bbox

tensor([[16., 13., 34., 46.],
        [ 3., 16., 15., 37.]])

### Resizing
On utilise le SSD300 donc son architecture est adaptee a des images `300x300`.

In [58]:
image = skimage.transform.resize(image_aug, (300, 300))
image.shape

(300, 300)

In [59]:
# Format de base de pytorch : (Taille_batch, Channels, Widht, Height)
image = np.moveaxis(image, -1, 0)
image = torch.FloatTensor(image)
image.shape, bbox

(torch.Size([300, 300]), tensor([[16., 13., 34., 46.],
         [ 3., 16., 15., 37.]]))

## Fin du preprocessing
Les images et les bounding boxes vont etre concatenees dans un batch et celui-ci sera passe a travers le `model()`.

L'architecture du model se trouve dans `model.py`.